In [ ]:
import pandas as pd
import folium
from datetime import datetime, timedelta, date
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import os

def find_project_root():
    """Find project root by looking for the data folder."""
    current_dir = os.getcwd()
    while current_dir != os.path.dirname(current_dir):  # Until filesystem root
        data_path = os.path.join(current_dir, 'data')
        if os.path.exists(data_path):
            return current_dir
        current_dir = os.path.dirname(current_dir)
    return os.getcwd()

class BudapestRouteTimelineVisualizer:
    def __init__(self, data_folder=None):
        """
        Enhanced Budapest route visualizer with timeline and shape variants
        """
        if data_folder is None:
            project_root = find_project_root()
            data_folder = os.path.join(project_root, 'data', 'processed')
        
        self.data_folder = data_folder
        self.load_data()
        self.setup_widgets()
        
    def load_data(self):
        """Load all required CSV files"""
        # Required files
        self.routes = pd.read_csv(os.path.join(self.data_folder, 'routes.csv'))
        self.route_versions = pd.read_csv(os.path.join(self.data_folder, 'route_versions.csv'))
        self.shapes = pd.read_csv(os.path.join(self.data_folder, 'shapes.csv'))
        
        # Optional files
        try:
            self.shape_variants = pd.read_csv(os.path.join(self.data_folder, 'shape_variants.csv'))
        except FileNotFoundError:
            self.shape_variants = pd.DataFrame()
            
        try:
            self.shape_variant_activations = pd.read_csv(os.path.join(self.data_folder, 'shape_variant_activations.csv'))
        except FileNotFoundError:
            self.shape_variant_activations = pd.DataFrame()
        
        # Load processing history to get last successful date
        try:
            import json
            processing_history_path = os.path.join(self.data_folder, 'processing_history.json')
            with open(processing_history_path, 'r') as f:
                processing_history = json.load(f)
                self.last_successful_date = pd.to_datetime(processing_history.get('last_successful_date')).date()
        except (FileNotFoundError, KeyError, TypeError):
            # Fallback to today's date if processing_history.json not found or invalid
            self.last_successful_date = date.today()
        
        # Convert date columns
        date_columns = ['valid_from', 'valid_to', 'date']
        for df in [self.route_versions, self.shape_variant_activations]:
            if not df.empty:
                for col in date_columns:
                    if col in df.columns:
                        df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # Initialize map state
        self.current_bounds = None
        self.current_zoom = 12
        
    def setup_widgets(self):
        """Setup all interactive widgets"""
        # Route selector
        route_options = []
        for _, row in self.routes.iterrows():
            route_id = row['route_id']
            short_name = row.get('route_short_name', str(route_id))
            long_name = row.get('route_long_name', '')
            
            display_name = f"{short_name}" + (f" - {long_name}" if long_name else "")
            route_options.append((display_name, route_id))
        
        self.route_selector = widgets.Dropdown(
            options=route_options,
            value=route_options[0][1] if route_options else None,
            description='Route:',
            style={'description_width': 'initial'}
        )
        
        # Date slider - will be updated when route is selected
        today = date.today()
        self.date_slider = widgets.SelectionSlider(
            options=[today],
            value=today,
            description='Date:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            style={'description_width': 'initial'}
        )
        
        # Show shape variants toggle
        self.show_variants_toggle = widgets.Checkbox(
            value=False,
            description='Show Shape Variants',
            disabled=False,
            style={'description_width': 'initial'}
        )
        
        # Show deleted lines toggle
        self.show_deleted_toggle = widgets.Checkbox(
            value=False,
            description='Show Deleted Lines',
            disabled=False,
            style={'description_width': 'initial'}
        )
        
        # Summary display toggles
        self.show_route_summary_toggle = widgets.Checkbox(
            value=True,
            description='Show Route Change Summary',
            disabled=False,
            style={'description_width': 'initial'}
        )
        
        self.show_variants_summary_toggle = widgets.Checkbox(
            value=False,
            description='Show Shape Variants Summary',
            disabled=False,
            style={'description_width': 'initial'}
        )
        
        self.show_deleted_summary_toggle = widgets.Checkbox(
            value=False,
            description='Show Deleted Lines Summary',
            disabled=False,
            style={'description_width': 'initial'}
        )
        
        # Set up event handlers
        self.route_selector.observe(self.on_route_change, names='value')
        self.date_slider.observe(self.on_date_change, names='value')
        self.show_variants_toggle.observe(self.on_toggle_change, names='value')
        self.show_deleted_toggle.observe(self.on_toggle_change, names='value')
        self.show_route_summary_toggle.observe(self.on_toggle_change, names='value')
        self.show_variants_summary_toggle.observe(self.on_toggle_change, names='value')
        self.show_deleted_summary_toggle.observe(self.on_toggle_change, names='value')
        
        # Initialize with first route
        if route_options:
            self.on_route_change({'new': route_options[0][1]})
    
    def get_route_timeline(self, route_id):
        """Get the full timeline for a route with daily granularity, limited by last_successful_date"""
        route_data = self.route_versions[self.route_versions['route_id'] == route_id]
        
        if route_data.empty:
            return [date.today()]
        
        # Get overall date range, handling NaT values
        valid_from_dates = route_data['valid_from'].dropna()
        valid_to_dates = route_data['valid_to'].dropna()
        
        if valid_from_dates.empty:
            return [date.today()]
        
        start_date = valid_from_dates.min().date()
        
        # Use last_successful_date as the effective end date
        if valid_to_dates.empty:
            end_date = self.last_successful_date
        else:
            route_end_date = valid_to_dates.max().date()
            # Use the earlier of route end date or last successful date
            end_date = min(route_end_date, self.last_successful_date)
        
        # Limit timeline to reasonable length (max 2 years or 730 days)
        max_days = 730
        if (end_date - start_date).days > max_days:
            # If timeline is too long, sample dates or use monthly intervals
            dates = []
            current_date = start_date
            while current_date <= end_date:
                dates.append(current_date)
                # Add monthly intervals for long timelines
                current_date += timedelta(days=30)
            return dates
        else:
            # Generate daily dates for reasonable timelines
            dates = []
            current_date = start_date
            while current_date <= end_date:
                dates.append(current_date)
                current_date += timedelta(days=1)
            
            return dates if dates else [date.today()]
    
    def get_main_shape_for_date(self, route_id, target_date):
        """Get the main_shape_id for a specific route on a specific date"""
        route_data = self.route_versions[self.route_versions['route_id'] == route_id]
        
        if route_data.empty:
            return None
        
        # Convert target_date to datetime
        target_datetime = datetime.combine(target_date, datetime.min.time())
        
        # Try to find active version for the target date with valid dates
        active_version = route_data[
            (route_data['valid_from'].notna()) & 
            (route_data['valid_to'].notna()) &
            (route_data['valid_from'] <= target_datetime) & 
            (route_data['valid_to'] >= target_datetime)
        ]
        
        # If no exact match found, try fallback approaches
        if active_version.empty:
            # Fallback 1: Check if there's a version with valid_from <= target_date and valid_to is NaT (ongoing)
            ongoing_version = route_data[
                (route_data['valid_from'].notna()) & 
                (route_data['valid_to'].isna()) &
                (route_data['valid_from'] <= target_datetime)
            ]
            
            if not ongoing_version.empty:
                # Take the latest starting version
                active_version = ongoing_version.sort_values('valid_from', ascending=False).head(1)
            else:
                # Fallback 2: Check if there's a version with valid_from is NaT and valid_to >= target_date
                starting_version = route_data[
                    (route_data['valid_from'].isna()) & 
                    (route_data['valid_to'].notna()) &
                    (route_data['valid_to'] >= target_datetime)
                ]
                
                if not starting_version.empty:
                    active_version = starting_version.head(1)
                else:
                    # Fallback 3: If still nothing, just take any version (routes with no date constraints)
                    if not route_data.empty:
                        active_version = route_data.head(1)
        
        if not active_version.empty:
            main_shape_id = active_version.iloc[0]['main_shape_id']
            return main_shape_id
            
        return None
    
    def get_shape_coordinates(self, shape_id):
        """Get coordinates for a specific shape"""
        if shape_id is None:
            return []
        
        shape_data = self.shapes[self.shapes['shape_id'] == shape_id].sort_values('shape_pt_sequence')
        coordinates = [(row['shape_pt_lat'], row['shape_pt_lon']) for _, row in shape_data.iterrows()]
        return coordinates
    
    def darken_color(self, hex_color, factor=0.3):
        """Darken a hex color by a given factor (0.0 = no change, 1.0 = black)"""
        # Remove # if present
        hex_color = hex_color.lstrip('#')
        
        # Convert to RGB
        rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        
        # Darken each component
        darkened_rgb = tuple(int(c * (1 - factor)) for c in rgb)
        
        # Convert back to hex
        return f"#{darkened_rgb[0]:02x}{darkened_rgb[1]:02x}{darkened_rgb[2]:02x}"
    def get_route_color(self, route_id):
        """Get the color for a specific route"""
        route_info = self.routes[self.routes['route_id'] == route_id]
        if route_info.empty:
            return '#0000FF'
        
        route_info = route_info.iloc[0]
        color = route_info.get('route_color', '#0000FF')
        
        if pd.notna(color):
            color = str(color)
            if not color.startswith('#'):
                color = '#' + color
            return color
        
        return '#0000FF'
    
    def get_active_shape_variants(self, route_id, target_date):
        """Get all active shape variants for a route on a specific date following the correct DB relationships"""
        if self.shape_variants.empty or self.shape_variant_activations.empty:
            return []
        
        # Convert target_date to datetime
        target_datetime = datetime.combine(target_date, datetime.min.time())
        
        # Step 1: Get the active version_id for this route on the target date
        route_data = self.route_versions[self.route_versions['route_id'] == route_id]
        
        active_version = route_data[
            (route_data['valid_from'].notna()) & 
            (route_data['valid_to'].notna()) &
            (route_data['valid_from'] <= target_datetime) & 
            (route_data['valid_to'] >= target_datetime)
        ]
        
        if active_version.empty:
            return []
        
        active_version_id = active_version.iloc[0]['version_id']
        
        # Step 2: Get all shape_variant_id-s that belong to this version_id
        version_variants = self.shape_variants[self.shape_variants['version_id'] == active_version_id]
        
        if version_variants.empty:
            return []
        
        active_variants = []
        
        # Step 3: For each shape_variant_id, check if it's active on the target date
        for _, variant in version_variants.iterrows():
            shape_variant_id = variant['shape_variant_id']
            
            # Step 4: Check activations for this shape_variant_id on the target date
            activations = self.shape_variant_activations[
                (self.shape_variant_activations['shape_variant_id'] == shape_variant_id) &
                (self.shape_variant_activations['date'].notna()) &
                (self.shape_variant_activations['date'].dt.date == target_date)
            ]
            
            for _, activation in activations.iterrows():
                exception_type = activation.get('exception_type', None)
                
                # Check if exception_type is NaN or 1 (active)
                if pd.isna(exception_type) or exception_type == 1:
                    variant_info = {
                        'shape_variant_id': shape_variant_id,
                        'version_id': active_version_id,
                        'shape_id': variant['shape_id'],
                        'trip_headsign': variant.get('trip_headsign', ''),
                        'is_main': variant.get('is_main', False),
                        'note': variant.get('note', ''),
                        'exception_type': exception_type,
                        'activation_date': activation['date']
                    }
                    active_variants.append(variant_info)
                    break  # Found active activation for this variant
        
        return active_variants
    
    def get_deleted_shape_variants(self, route_id, target_date):
        """Get all deleted shape variants for a route on a specific date (exception_type = 2)"""
        if self.shape_variants.empty or self.shape_variant_activations.empty:
            return []
        
        # Convert target_date to datetime
        target_datetime = datetime.combine(target_date, datetime.min.time())
        
        # Step 1: Get the active version_id for this route on the target date
        route_data = self.route_versions[self.route_versions['route_id'] == route_id]
        
        active_version = route_data[
            (route_data['valid_from'].notna()) & 
            (route_data['valid_to'].notna()) &
            (route_data['valid_from'] <= target_datetime) & 
            (route_data['valid_to'] >= target_datetime)
        ]
        
        if active_version.empty:
            return []
        
        active_version_id = active_version.iloc[0]['version_id']
        
        # Step 2: Get all shape_variant_id-s that belong to this version_id
        version_variants = self.shape_variants[self.shape_variants['version_id'] == active_version_id]
        
        if version_variants.empty:
            return []
        
        deleted_variants = []
        
        # Step 3: For each shape_variant_id, check if it's deleted on the target date (exception_type = 2)
        for _, variant in version_variants.iterrows():
            shape_variant_id = variant['shape_variant_id']
            
            # Step 4: Check activations for this shape_variant_id on the target date
            activations = self.shape_variant_activations[
                (self.shape_variant_activations['shape_variant_id'] == shape_variant_id) &
                (self.shape_variant_activations['date'].notna()) &
                (self.shape_variant_activations['date'].dt.date == target_date)
            ]
            
            for _, activation in activations.iterrows():
                exception_type = activation.get('exception_type', None)
                
                # Check if exception_type is 2 (deleted)
                if pd.notna(exception_type) and exception_type == 2:
                    variant_info = {
                        'shape_variant_id': shape_variant_id,
                        'version_id': active_version_id,
                        'shape_id': variant['shape_id'],
                        'trip_headsign': variant.get('trip_headsign', ''),
                        'is_main': variant.get('is_main', False),
                        'note': variant.get('note', ''),
                        'exception_type': exception_type,
                        'activation_date': activation['date']
                    }
                    deleted_variants.append(variant_info)
                    break  # Found deleted activation for this variant
        
        return deleted_variants
    def create_map(self, route_id, target_date, show_variants=False, show_deleted=False):
        """Create the main map visualization"""
        # Get main shape
        main_shape_id = self.get_main_shape_for_date(route_id, target_date)
        main_coordinates = self.get_shape_coordinates(main_shape_id)
        
        # Get route info
        route_info = self.routes[self.routes['route_id'] == route_id]
        route_color = self.get_route_color(route_id)
        
        if route_info.empty:
            route_name = f"Route {route_id}"
        else:
            route_info = route_info.iloc[0]
            short_name = route_info.get('route_short_name', str(route_id))
            long_name = route_info.get('route_long_name', '')
            route_name = f"{short_name}" + (f" - {long_name}" if long_name else "")
        
        # Create map
        if not main_coordinates:
            m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)
            folium.Marker(
                [47.4979, 19.0402],
                popup="No route data available for this date",
                icon=folium.Icon(color='red')
            ).add_to(m)
            return m
        
        # Calculate center
        center_lat = sum(coord[0] for coord in main_coordinates) / len(main_coordinates)
        center_lon = sum(coord[1] for coord in main_coordinates) / len(main_coordinates)
        
        # Create map with preserved zoom
        if self.current_bounds:
            m = folium.Map(location=[center_lat, center_lon])
            m.fit_bounds(self.current_bounds)
        else:
            m = folium.Map(location=[center_lat, center_lon], zoom_start=self.current_zoom)
        
        # Add main route line
        folium.PolyLine(
            main_coordinates,
            color=route_color,
            weight=8,
            opacity=0.8,
            popup=f"{route_name} - Main Route on {target_date}"
        ).add_to(m)
        
        # Add shape variants if requested
        if show_variants:
            active_variants = self.get_active_shape_variants(route_id, target_date)
            variant_color = self.darken_color(route_color, 0.3)  # Darker version of main color
            
            for i, variant in enumerate(active_variants):
                # Skip variants where is_main is True since main shape is already shown
                if variant['is_main']:
                    continue
                    
                variant_coordinates = self.get_shape_coordinates(variant['shape_id'])
                if variant_coordinates:
                    folium.PolyLine(
                        variant_coordinates,
                        color=variant_color,
                        weight=4,
                        opacity=0.6,
                        popup=f"Variant {variant['shape_variant_id']}: {variant['trip_headsign']}"
                    ).add_to(m)
        
        # Add deleted lines if requested
        if show_deleted:
            deleted_variants = self.get_deleted_shape_variants(route_id, target_date)
            
            for i, variant in enumerate(deleted_variants):
                # Skip variants where is_main is True since main shape is already shown
                if variant['is_main']:
                    continue
                    
                variant_coordinates = self.get_shape_coordinates(variant['shape_id'])
                if variant_coordinates:
                    folium.PolyLine(
                        variant_coordinates,
                        color='#FF0000',  # Red color for deleted lines
                        weight=3,
                        opacity=0.5,
                        dashArray='5, 10',  # Dashed line style
                        popup=f"DELETED: {variant['shape_variant_id']}: {variant['trip_headsign']}"
                    ).add_to(m)
        
        # Add start/end markers for main route
        if len(main_coordinates) >= 2:
            folium.Marker(
                main_coordinates[0],
                popup=f"Start: {route_name}",
                icon=folium.Icon(color='green', icon='play')
            ).add_to(m)
            
            folium.Marker(
                main_coordinates[-1],
                popup=f"End: {route_name}",
                icon=folium.Icon(color='red', icon='stop')
            ).add_to(m)
        
        return m
    
    def get_route_change_summary(self, route_id):
        """Generate a summary of all route changes throughout its lifetime"""
        route_data = self.route_versions[self.route_versions['route_id'] == route_id].copy()
        
        if route_data.empty:
            return "No route version data available."
        
        # Sort by valid_from date
        route_data = route_data.sort_values('valid_from')
        
        # Get route basic info
        route_info = self.routes[self.routes['route_id'] == route_id]
        if not route_info.empty:
            route_info = route_info.iloc[0]
            short_name = route_info.get('route_short_name', str(route_id))
            long_name = route_info.get('route_long_name', '')
            route_name = f"{short_name}" + (f" - {long_name}" if long_name else "")
            route_color = self.get_route_color(route_id)
        else:
            route_name = f"Route {route_id}"
            route_color = "#0000FF"
        
        summary = []
        summary.append(f"🚊 **{route_name} - Lifetime Summary**")
        summary.append(f"🎨 Color: {route_color}")
        summary.append(f"📊 Total Versions: {len(route_data)}")
        
        if len(route_data) > 0:
            lifetime_start = route_data['valid_from'].min().strftime('%Y-%m-%d') if pd.notna(route_data['valid_from'].min()) else 'Unknown'
            lifetime_end = route_data['valid_to'].max().strftime('%Y-%m-%d') if pd.notna(route_data['valid_to'].max()) else 'Present'
            summary.append(f"⏰ Lifetime: {lifetime_start} → {lifetime_end}")
        
        summary.append(f"\n📅 **Detailed Change History:**")
        
        for i, (_, version) in enumerate(route_data.iterrows(), 1):
            start_date = version['valid_from'].strftime('%Y-%m-%d') if pd.notna(version['valid_from']) else 'Unknown'
            end_date = version['valid_to'].strftime('%Y-%m-%d') if pd.notna(version['valid_to']) else 'Present'
            main_shape = version['main_shape_id']
            direction = version.get('direction_id', 'Unknown')
            version_id = version.get('version_id', 'Unknown')
            
            summary.append(f"  {i}. {start_date} → {end_date}")
            summary.append(f"     📍 Shape: {main_shape} | Direction: {direction} | Version: {version_id}")
            
            # Add route description if available
            route_desc = version.get('route_desc', '')
            if route_desc:
                summary.append(f"     📝 Description: {route_desc}")
        
        return "\\n".join(summary)

    def on_route_change(self, change):
        """Handle route selection change"""
        route_id = change['new']
        
        # Update date slider with route timeline
        timeline = self.get_route_timeline(route_id)
        if timeline:
            self.date_slider.options = timeline
            self.date_slider.value = timeline[0]
        
        self.update_display()
    
    def on_date_change(self, change):
        """Handle date change"""
        self.update_display()
    
    def on_toggle_change(self, change):
        """Handle show variants/deleted toggle change"""
        self.update_display()
    
    def update_display(self):
        """Update all display elements"""
        # This will be called by the display method
        pass
    
    def get_shape_variants_summary(self, route_id, target_date):
        """Generate summary of active shape variants for the selected date"""
        active_variants = self.get_active_shape_variants(route_id, target_date)
        
        if not active_variants:
            return f"🔀 **Shape Variants on {target_date}:** None found"
        
        summary = []
        summary.append(f"🔀 **Active Shape Variants on {target_date} ({len(active_variants)} variants):**")
        
        for i, variant in enumerate(active_variants, 1):
            shape_variant_id = variant['shape_variant_id']
            shape_id = variant['shape_id']
            trip_headsign = variant['trip_headsign']
            is_main = variant['is_main']
            note = variant['note']
            exception_type = variant['exception_type']
            
            # Format exception type
            exception_str = 'NaN' if pd.isna(exception_type) else str(int(exception_type))
            main_indicator = " (MAIN)" if is_main else ""
            
            summary.append(f"  {i}. Variant ID: {shape_variant_id}{main_indicator}")
            summary.append(f"     🎯 Shape: {shape_id}")
            summary.append(f"     🚏 Headsign: {trip_headsign}")
            summary.append(f"     ⚡ Exception Type: {exception_str}")
            
            if note:
                summary.append(f"     📝 Note: {note}")
        
        return "\\n".join(summary)
    
    def get_deleted_shape_variants_summary(self, route_id, target_date):
        """Generate summary of deleted shape variants for the selected date"""
        deleted_variants = self.get_deleted_shape_variants(route_id, target_date)
        
        if not deleted_variants:
            return f"🗑️ **Deleted Lines on {target_date}:** None found"
        
        summary = []
        summary.append(f"🗑️ **Deleted Lines on {target_date} ({len(deleted_variants)} deleted):**")
        
        for i, variant in enumerate(deleted_variants, 1):
            shape_variant_id = variant['shape_variant_id']
            shape_id = variant['shape_id']
            trip_headsign = variant['trip_headsign']
            is_main = variant['is_main']
            note = variant['note']
            
            main_indicator = " (MAIN)" if is_main else ""
            
            summary.append(f"  {i}. Variant ID: {shape_variant_id}{main_indicator}")
            summary.append(f"     🎯 Shape: {shape_id}")
            summary.append(f"     🚏 Headsign: {trip_headsign}")
            summary.append(f"     ❌ Status: DELETED (Exception Type: 2)")
            
            if note:
                summary.append(f"     📝 Note: {note}")
        
        return "\\n".join(summary)
        """Handle route selection change"""
        route_id = change['new']
        
        # Update date slider with route timeline
        timeline = self.get_route_timeline(route_id)
        if timeline:
            self.date_slider.options = timeline
            self.date_slider.value = timeline[0]
        
        self.update_display()
    
    def on_date_change(self, change):
        """Handle date change"""
        self.update_display()
    
    def on_toggle_change(self, change):
        """Handle show variants toggle change"""
        self.update_display()
    
    def update_display(self):
        """Update all display elements"""
        # This will be called by the display method
        pass
    
    def display(self):
        """Display the complete interactive visualization"""
        # Control widgets - organized in sections
        map_controls = widgets.VBox([
            widgets.HTML("<h4>🗺️ Map Controls</h4>"),
            self.route_selector,
            self.date_slider,
            self.show_variants_toggle,
            self.show_deleted_toggle
        ])
        
        summary_controls = widgets.VBox([
            widgets.HTML("<h4>📊 Summary Controls</h4>"),
            self.show_route_summary_toggle,
            self.show_variants_summary_toggle,
            self.show_deleted_summary_toggle
        ])
        
        controls = widgets.HBox([map_controls, summary_controls])
        display(controls)
        
        # Map output
        map_output = widgets.Output()
        display(map_output)
        
        # Summary outputs
        route_summary_output = widgets.Output()
        variants_summary_output = widgets.Output()
        deleted_summary_output = widgets.Output()
        
        # Create conditional display containers
        route_summary_container = widgets.VBox([
            widgets.HTML("<h3>📊 Route Change Summary</h3>"),
            route_summary_output
        ])
        
        variants_summary_container = widgets.VBox([
            widgets.HTML("<h3>🔀 Shape Variants Summary</h3>"),
            variants_summary_output
        ])
        
        deleted_summary_container = widgets.VBox([
            widgets.HTML("<h3>🗑️ Deleted Lines Summary</h3>"),
            deleted_summary_output
        ])
        
        summaries_container = widgets.VBox([
            route_summary_container,
            variants_summary_container,
            deleted_summary_container
        ])
        
        display(summaries_container)
        
        def update_all(*args):
            route_id = self.route_selector.value
            target_date = self.date_slider.value
            show_variants = self.show_variants_toggle.value
            show_deleted = self.show_deleted_toggle.value
            
            # Summary display flags
            show_route_summary = self.show_route_summary_toggle.value
            show_variants_summary = self.show_variants_summary_toggle.value
            show_deleted_summary = self.show_deleted_summary_toggle.value
            
            if not route_id:
                return
            
            # Update map
            with map_output:
                clear_output(wait=True)
                map_obj = self.create_map(route_id, target_date, show_variants, show_deleted)
                display(map_obj)
            
            # Show/hide summary containers
            route_summary_container.layout.display = 'block' if show_route_summary else 'none'
            variants_summary_container.layout.display = 'block' if show_variants_summary else 'none'
            deleted_summary_container.layout.display = 'block' if show_deleted_summary else 'none'
            
            # Update route summary (only if visible)
            if show_route_summary:
                with route_summary_output:
                    clear_output(wait=True)
                    route_summary = self.get_route_change_summary(route_id)
                    for line in route_summary.split('\\n'):
                        print(line)
            else:
                with route_summary_output:
                    clear_output(wait=True)
            
            # Update variants summary (only if visible)
            if show_variants_summary:
                with variants_summary_output:
                    clear_output(wait=True)
                    if show_variants:
                        variants_summary = self.get_shape_variants_summary(route_id, target_date)
                        for line in variants_summary.split('\\n'):
                            print(line)
                    else:
                        print("🔀 Shape variants display on map is OFF. Enable 'Show Shape Variants' to see variant details.")
            else:
                with variants_summary_output:
                    clear_output(wait=True)
            
            # Update deleted lines summary (only if visible)
            if show_deleted_summary:
                with deleted_summary_output:
                    clear_output(wait=True)
                    if show_deleted:
                        deleted_summary = self.get_deleted_shape_variants_summary(route_id, target_date)
                        for line in deleted_summary.split('\\n'):
                            print(line)
                    else:
                        print("🗑️ Deleted lines display on map is OFF. Enable 'Show Deleted Lines' to see deleted line details.")
            else:
                with deleted_summary_output:
                    clear_output(wait=True)
        
        # Connect all event handlers
        self.route_selector.observe(update_all, names='value')
        self.date_slider.observe(update_all, names='value')
        self.show_variants_toggle.observe(update_all, names='value')
        self.show_deleted_toggle.observe(update_all, names='value')
        self.show_route_summary_toggle.observe(update_all, names='value')
        self.show_variants_summary_toggle.observe(update_all, names='value')
        self.show_deleted_summary_toggle.observe(update_all, names='value')
        
        # Initial display
        update_all()

# Usage:
visualizer = BudapestRouteTimelineVisualizer()
visualizer.display()

Output()